In [ ]:
import logging
import matplotlib.pyplot as plt

from analyzer import GaiaAnalyzer
from analyzer import MISTFitter, BinaryMixtureFitter
from downloader import LightCurveExtractor

%load_ext autoreload
%autoreload 2

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [ ]:
# some statements to make the figures look nicer 
%config InlineBackend.figure_format='retina'
plt.rcParams.update({'axes.linewidth' : 1,
                     'ytick.major.width' : 1,
                     'ytick.minor.width' : 1,
                     'xtick.major.width' : 1,
                     'xtick.minor.width' : 1,
                     'xtick.labelsize': 10, 
                     'ytick.labelsize': 10,
                     'axes.labelsize': 12,
                    
                     'font.family': 'Serif',
                     'figure.figsize': (10, 6)
                    })

In [ ]:
gaia_analyzer = GaiaAnalyzer(filename='gaia.csv')
gaia_analyzer.get_data()

df = gaia_analyzer.get_data().copy()
df.head()

In [ ]:
gaia_analyzer.plot_hr_diagram(show=True)

In [ ]:
fitter = MISTFitter(data=df, age_range=(70e6, 160e6))

In [ ]:
sampler = fitter.sample_cluster(n_walkers=32, n_burn=200, n_steps=2000)

In [ ]:
# Step 3: Inspect posterior
samples = fitter.get_samples()
median_params = fitter.get_median_model()
print(f"Median cluster parameters:\nAge={median_params[0]:.2e} yr, [Fe/H]={median_params[1]:.2f}, "
      f"Distance={median_params[2]:.1f} pc, A_V={median_params[3]:.2f}")

# Step 4: Plot median
fitter.plot_isochrone(median_params)

In [ ]:
# Step 5: plot best model
best_params = fitter.get_best_model()
print(f"Best cluster parameters:\nAge={median_params[0]:.2e} yr, [Fe/H]={median_params[1]:.2f}, "
      f"Distance={median_params[2]:.1f} pc, A_V={median_params[3]:.2f}")

# Step 4: Plot median
fitter.plot_isochrone(best_params)

In [ ]:
fitter.plot_good_isochrones(chi2_cutoff=1)

In [ ]:
fitter.plot_residuals(theta=fitter.get_best_model())

In [ ]:
fitter.plot_corner(sampler, discard=200, thin=200)

In [ ]:
# Testing using actual isochrone
import pandas as pd

isochrone_df = pd.read_csv('mist_isochrone_140Myr.dat', delim_whitespace=True, comment='#')
fig, ax = gaia_analyzer.plot_hr_diagram(show=False)

isochrone_df['color'] = isochrone_df['Gaia_BP_EDR3'] - isochrone_df['Gaia_RP_EDR3'] + 0.15
isochrone_df['magnitude'] = isochrone_df['Gaia_G_EDR3'] + 6

ax.plot(isochrone_df['color'], isochrone_df['magnitude'], color='green', label='Isochrone')
plt.show()

In [ ]:
from analyzer import BinaryMixtureFitter

binary_fitter = BinaryMixtureFitter(data=df)
binary_fitter.fit(theta=fitter.get_best_model())

print(binary_fitter.get_mixture_weights())

In [ ]:
from downloader import LightCurveExtractor

lightcurve_analyzer = LightCurveExtractor(download_dir='lightcurves/')

ids = gaia_analyzer.extract_source_ids()[:10]
lightcurves = lightcurve_analyzer.extract_gaia_lightcurves(ids)

In [ ]:
lightcurves

In [ ]:
from analyzer import TESSAnalyzer

tess_analyzer = TESSAnalyzer(lightcurves=lightcurves)
tess_analyzer.batch_plot()

In [ ]:
results = tess_analyzer.batch_analyze()
tess_analyzer.batch_plot_folded()

In [ ]:
import matplotlib.pyplot as plt

R = binary_fitter.E_step(binary_fitter.theta)
labels = ["Single", "Binary", "Field"]
colors = [R[:,0], R[:,1], R[:,2]]

fig, axes = plt.subplots(1, 3, figsize=(15, 8), sharex=True, sharey=True)
for ax, c, lbl in zip(axes, colors, labels):
    sc = ax.scatter(binary_fitter.BP - binary_fitter.RP, binary_fitter.G,
                    c=c, cmap="plasma", s=10)
    ax.invert_yaxis()
    ax.set_title(lbl)
    plt.colorbar(sc, ax=ax, label=f"P({lbl})")
plt.xlabel("(BP − RP)")
plt.ylabel("G magnitude")
plt.tight_layout()
plt.show()
